# Player Allocation

Upon completion of evaluation of overall performance for all skaters, each player will be categorized and assigned to their respectful position within the roster of their team. The fundamental statistic used to differentiate top from bottom players, is zone start percentage. There are three zones from where plays can be started: offensive, neutral and defensive. Offensive zone start percentage is calculated by the number of face-offs held in the attacking area divided by the sum of face-offs an individual player was on the ice for. Identically, defensive zone start percentage is computed by the number of face-offs taken in their own territory divided by the total face-offs each player was on the ice for. Skaters who are talented in creating opportunities and producing goals will have a greater percentage of offensive zone starts in comparison to defensive zone starts. Equivalently, players who are skilled in preventing chances and goals being conceded, will have a higher defensive zone start percentage in correlation to offensive zone starts. In other words, top six forwards and top four pairing defensemen will start much of their shifts in the zone their opponent whereas bottom six forwards and bottom pairing defensemen in their own zone.


### purpose of notebook:

a) create two variables: offensive zone start and defensive zone start.

b) sum up the total offensive and defensive zone start per player.

c) categorize forwards into top and bottom six.

d) categorize defensemen into top four and bottom pairing.

e) determine each player's roster depth position

##  import modules

In [189]:
import sys
import os
import pandas as pd
import numpy as np
import datetime, time
import matplotlib.pyplot as plt
import statsmodels.api as sm
from statsmodels.formula.api import ols
from pylab import hist, show
import scipy

## import data frame

The player evaluation data frame is used for player allocation.

In [190]:
dm = pd.read_csv('plyreval.csv')

## drop unnamed column (irrelevant)

In [191]:
dm = dm.drop('Unnamed: 0', axis=1)

## zone start

With the help of zone variable, offensive, neutral and defensive zone starts will be created.

**zone start variable:** 

- a value of 1 will be assigned if the on-ice event happened in the offensive zone.

- a value of 0 will be assigned if the on-ice event happened in the neutral zone.

- a value of -1 if it happened in the defensive zone of the representative team.

In [192]:
dm['zs'] = np.where(dm['Zone'] == 'O', 1,
                    (np.where(dm['Zone'] == 'D', -1, 0)))

## home and visitor zone start

- visitor team zone start (vzs)

If team code of event is the same as visitor team, the visitor zone start variable will be assigned identical value to zone start. If not, it will be assigned the opposite (negative) value of zone start. 

In [193]:
dm['vzs'] = np.where(dm['TeamCode'] == dm['VTeamCode'], dm['zs'], -dm['zs'] )

- home team zone start (hzs) 

If team code of event is the same as home team, the home team will be assigned identical value to zone start. If not, it will be assigned the opposite (negative) value of zone start. 

In [194]:
dm['hzs'] = np.where(dm['TeamCode'] == dm['HTeamCode'], dm['zs'], -dm['zs'] )

## assign zone start to players

The value of zone start is assigned to all players that were on ice, a total of 12 players (6 per team). The overall zone start variable of each player is the total (sum) of events they participated in. 

### a) overall zone start of each player from the visitor team in all (6) positions 

Group data frame by season, visitor team code and visitor player position to seperate players that play in the same position. 

- create variable that sums up the overall zone start of each player from the visitor team that is listed as "VPlayer1"

In [195]:
dm['zvp1'] = dm.groupby(['Season', 'VTeamCode', 'VPlayer1'])['vzs'].transform('sum')

- create variable that sums up the overall zone start of each player from the visitor team that is listed as "VPlayer2"

In [196]:
dm['zvp2'] = dm.groupby(['Season', 'VTeamCode', 'VPlayer2'])['vzs'].transform('sum')

- create variable that sums up the overall zone start of each player from the visitor team that is listed as "VPlayer3"

In [197]:
dm['zvp3'] = dm.groupby(['Season', 'VTeamCode', 'VPlayer3'])['vzs'].transform('sum')

- create variable that sums up the overall zone start of each player from the visitor team that is listed as "VPlayer4"

In [198]:
dm['zvp4'] = dm.groupby(['Season', 'VTeamCode', 'VPlayer4'])['vzs'].transform('sum')

- create variable that sums up the overall zone start of each player from the visitor team that is listed as "VPlayer5"

In [199]:
dm['zvp5'] = dm.groupby(['Season', 'VTeamCode', 'VPlayer5'])['vzs'].transform('sum')

- create variable that sums up the overall zone start of each player from the visitor team that is listed as "VPlayer6"

In [200]:
dm['zvp6'] = dm.groupby(['Season', 'VTeamCode', 'VPlayer6'])['vzs'].transform('sum')

### b) overall zone start of each player from the home team in all (6) positions

Group data frame by season, home team code and home player position to seperate players that play in the same position.

- create variable that sums up the overall zone start of each player from the visitor team that is listed as "HPlayer1"

In [201]:
dm['zhp1'] = dm.groupby(['Season', 'HTeamCode', 'HPlayer1'])['hzs'].transform('sum')

- create variable that sums up the overall zone start of each player from the visitor team that is listed as "HPlayer2"

In [202]:
dm['zhp2'] = dm.groupby(['Season', 'HTeamCode', 'HPlayer2'])['hzs'].transform('sum')

- create variable that sums up the overall zone start of each player from the visitor team that is listed as "HPlayer3"

In [203]:
dm['zhp3'] = dm.groupby(['Season', 'HTeamCode', 'HPlayer3'])['hzs'].transform('sum')

- create variable that sums up the overall zone start of each player from the visitor team that is listed as "HPlayer4"

In [204]:
dm['zhp4'] = dm.groupby(['Season', 'HTeamCode', 'HPlayer4'])['hzs'].transform('sum')

- create variable that sums up the overall zone start of each player from the visitor team that is listed as "HPlayer5"

In [205]:
dm['zhp5'] = dm.groupby(['Season', 'HTeamCode', 'HPlayer5'])['hzs'].transform('sum')

- create variable that sums up the overall zone start of each player from the visitor team that is listed as "HPlayer6"

In [206]:
dm['zhp6'] = dm.groupby(['Season', 'HTeamCode', 'HPlayer6'])['hzs'].transform('sum')

## players that play in multiple positions

Throughout the duration of a game, a player may change position. As mentioned above, the overall impact of a given player is the total (sum) of events he participated in. To properly measure each player's contribution, a cross examination per position must be applied.

- **a) cross examine each position for visitor team:**

- if visitor player in position 1 played in any other position, the total of zone starts he was on the ice for are calculated with the use of **np.where**.

In [241]:
dm['tzvp1'] = np.where(dm['VPlayer1'] == dm['VPlayer2'], dm['zvp1'] + dm['zvp2'],
                     (np.where(dm['VPlayer1'] == dm['VPlayer3'], dm['zvp1'] + dm['zvp3'],
                              (np.where(dm['VPlayer1'] == dm['VPlayer4'], dm['zvp1'] + dm['zvp4'],
                                       (np.where(dm['VPlayer1'] == dm['VPlayer5'], dm['zvp1'] + dm['zvp5'],
                                                (np.where(dm['VPlayer1'] == dm['VPlayer6'], dm['zvp1'] + dm['zvp6'], dm['zvp1'])))))))))

- if visitor player in position 2 played in any other position, the total of zone starts he was on the ice for are calculated with the use of **np.where**. If visitor player in position 2, didn't play in any other position, the total of zone starts in position 2 he participated in, are listed.

In [208]:
dm['tzvp2'] = np.where(dm['VPlayer2'] == dm['VPlayer1'], dm['zvp2'] + dm['zvp1'],
                     (np.where(dm['VPlayer2'] == dm['VPlayer3'], dm['zvp2'] + dm['zvp3'],
                              (np.where(dm['VPlayer2'] == dm['VPlayer4'], dm['zvp2'] + dm['zvp4'],
                                       (np.where(dm['VPlayer2'] == dm['VPlayer5'], dm['zvp2'] + dm['zvp5'],
                                                (np.where(dm['VPlayer2'] == dm['VPlayer6'], dm['zvp2'] + dm['zvp6'], dm['zvp2'])))))))))

- if visitor player in position 3 played in any other position, the total of zone starts he was on the ice for are calculated with the use of **np.where**. If visitor player in position 3, didn't play in any other position, the total of zone starts in position 3 he participated in, are listed.

In [209]:
dm['tzvp3'] = np.where(dm['VPlayer3'] == dm['VPlayer1'], dm['zvp3'] + dm['zvp1'],
                     (np.where(dm['VPlayer3'] == dm['VPlayer2'], dm['zvp3'] + dm['zvp2'],
                              (np.where(dm['VPlayer3'] == dm['VPlayer4'], dm['zvp3'] + dm['zvp4'],
                                       (np.where(dm['VPlayer3'] == dm['VPlayer5'], dm['zvp3'] + dm['zvp5'],
                                                (np.where(dm['VPlayer3'] == dm['VPlayer6'], dm['zvp3'] + dm['zvp6'], dm['zvp3'])))))))))

- if visitor player in position 4 played in any other position, the total of zone starts he was on the ice for are calculated with the use of **np.where**. If visitor player in position 4, didn't play in any other position, the total of zone starts in position 4 he participated in, are listed.

In [210]:
dm['tzvp4'] = np.where(dm['VPlayer4'] == dm['VPlayer1'], dm['zvp4'] + dm['zvp1'],
                     (np.where(dm['VPlayer4'] == dm['VPlayer2'], dm['zvp4'] + dm['zvp2'],
                              (np.where(dm['VPlayer4'] == dm['VPlayer3'], dm['zvp4'] + dm['zvp3'],
                                       (np.where(dm['VPlayer4'] == dm['VPlayer5'], dm['zvp4'] + dm['zvp5'],
                                                (np.where(dm['VPlayer4'] == dm['VPlayer6'], dm['zvp1'] + dm['zvp6'], dm['zvp4'])))))))))

- if visitor player in position 5 played in any other position, the total of zone starts he was on the ice for are calculated with the use of **np.where**. If visitor player in position 5, didn't play in any other position, the total of zone starts in position 5 he participated in, are listed.

In [211]:
dm['tzvp5'] = np.where(dm['VPlayer5'] == dm['VPlayer1'], dm['zvp5'] + dm['zvp1'],
                     (np.where(dm['VPlayer5'] == dm['VPlayer2'], dm['zvp5'] + dm['zvp2'],
                              (np.where(dm['VPlayer5'] == dm['VPlayer3'], dm['zvp5'] + dm['zvp3'],
                                       (np.where(dm['VPlayer5'] == dm['VPlayer4'], dm['zvp5'] + dm['zvp4'],
                                                (np.where(dm['VPlayer5'] == dm['VPlayer6'], dm['zvp5'] + dm['zvp6'], dm['zvp5'])))))))))

- if visitor player in position 6 played in any other position, the total of zone starts he was on the ice for are calculated with the use of **np.where**. If visitor player in position 6, didn't play in any other position, the total of zone starts in position 6 he participated in, are listed.

In [212]:
dm['tzvp6'] = np.where(dm['VPlayer6'] == dm['VPlayer1'], dm['zvp6'] + dm['zvp1'],
                     (np.where(dm['VPlayer6'] == dm['VPlayer2'], dm['zvp6'] + dm['zvp2'],
                              (np.where(dm['VPlayer6'] == dm['VPlayer3'], dm['zvp6'] + dm['zvp3'],
                                       (np.where(dm['VPlayer6'] == dm['VPlayer4'], dm['zvp6'] + dm['zvp4'],
                                                (np.where(dm['VPlayer6'] == dm['VPlayer5'], dm['zvp6'] + dm['zvp5'], dm['zvp6'])))))))))

- **b) cross examine each position for home team:**

- if home player in position 1 played in any other position, the total of zone starts he was on the ice for are calculated with the use of **np.where**. If home player in position 1, didn't play in any other position, the total of zone starts in position 1 he participated in, are listed.

In [213]:
dm['tzhp1'] = np.where(dm['HPlayer1'] == dm['HPlayer2'], dm['zhp1'] + dm['zhp2'],
                     (np.where(dm['HPlayer1'] == dm['HPlayer3'], dm['zhp1'] + dm['zhp3'],
                              (np.where(dm['HPlayer1'] == dm['HPlayer4'], dm['zhp1'] + dm['zhp4'],
                                       (np.where(dm['HPlayer1'] == dm['HPlayer5'], dm['zhp1'] + dm['zhp5'],
                                                (np.where(dm['HPlayer1'] == dm['HPlayer6'], dm['zhp1'] + dm['zhp6'], dm['zhp1'])))))))))

- if home player in position 2 played in any other position, the total of zone starts he was on the ice for are calculated with the use of **np.where**. If home player in position 2, didn't play in any other position, the total of zone starts in position 2 he participated in, are listed.

In [214]:
dm['tzhp2'] = np.where(dm['HPlayer2'] == dm['HPlayer1'], dm['zhp2'] + dm['zhp1'],
                     (np.where(dm['HPlayer2'] == dm['HPlayer3'], dm['zhp2'] + dm['zhp3'],
                              (np.where(dm['HPlayer2'] == dm['HPlayer4'], dm['zhp2'] + dm['zhp4'],
                                       (np.where(dm['HPlayer2'] == dm['HPlayer5'], dm['zhp2'] + dm['zhp5'],
                                                (np.where(dm['HPlayer2'] == dm['HPlayer6'], dm['zhp2'] + dm['zhp6'], dm['zhp2'])))))))))

- if home player in position 3 played in any other position, the total of zone starts he was on the ice for are calculated with the use of **np.where**. If home player in position 3, didn't play in any other position, the total of zone starts in position 3 he participated in, are listed.

In [215]:
dm['tzhp3'] = np.where(dm['HPlayer3'] == dm['HPlayer1'], dm['zhp3'] + dm['zhp1'],
                     (np.where(dm['HPlayer3'] == dm['HPlayer2'], dm['zhp3'] + dm['zhp2'],
                              (np.where(dm['HPlayer3'] == dm['HPlayer4'], dm['zhp3'] + dm['zhp4'],
                                       (np.where(dm['HPlayer3'] == dm['HPlayer5'], dm['zhp3'] + dm['zhp5'],
                                                (np.where(dm['HPlayer3'] == dm['HPlayer6'], dm['zhp3'] + dm['zhp6'], dm['zhp3'])))))))))

- if home player in position 4 played in any other position, the total of zone starts he was on the ice for are calculated with the use of **np.where**. If home player in position 4, didn't play in any other position, the total of zone starts in position 4 he participated in, are listed.

In [216]:
dm['tzhp4'] = np.where(dm['HPlayer4'] == dm['HPlayer1'], dm['zhp4'] + dm['zhp1'],
                     (np.where(dm['HPlayer4'] == dm['HPlayer2'], dm['zhp4'] + dm['zhp2'],
                              (np.where(dm['HPlayer4'] == dm['HPlayer3'], dm['zhp4'] + dm['zhp3'],
                                       (np.where(dm['HPlayer4'] == dm['HPlayer5'], dm['zhp4'] + dm['zhp5'],
                                                (np.where(dm['HPlayer4'] == dm['HPlayer6'], dm['zhp1'] + dm['zhp6'], dm['zhp4'])))))))))

- if home player in position 5 played in any other position, the total of zone starts he was on the ice for are calculated with the use of **np.where**. If home player in position 5, didn't play in any other position, the total of zone starts in position 5 he participated in, are listed.

In [217]:
dm['tzhp5'] = np.where(dm['HPlayer5'] == dm['HPlayer1'], dm['zhp5'] + dm['zhp1'],
                     (np.where(dm['HPlayer5'] == dm['HPlayer2'], dm['zhp5'] + dm['zhp2'],
                              (np.where(dm['HPlayer5'] == dm['HPlayer3'], dm['zhp5'] + dm['zhp3'],
                                       (np.where(dm['HPlayer5'] == dm['HPlayer4'], dm['zhp5'] + dm['zhp4'],
                                                (np.where(dm['HPlayer5'] == dm['HPlayer6'], dm['zhp5'] + dm['zhp6'], dm['zhp5'])))))))))

- if home player in position 6 played in any other position, the total of zone starts he was on the ice for are calculated with the use of **np.where**. If home player in position 6, didn't play in any other position, the total of zone starts in position 6 he participated in, are listed.

In [218]:
dm['tzhp6'] = np.where(dm['HPlayer6'] == dm['HPlayer1'], dm['zhp6'] + dm['zhp1'],
                     (np.where(dm['HPlayer6'] == dm['HPlayer2'], dm['zhp6'] + dm['zhp2'],
                              (np.where(dm['HPlayer6'] == dm['HPlayer3'], dm['zhp6'] + dm['zhp3'],
                                       (np.where(dm['HPlayer6'] == dm['HPlayer4'], dm['zhp6'] + dm['zhp4'],
                                                (np.where(dm['HPlayer6'] == dm['HPlayer5'], dm['zhp6'] + dm['zhp5'], dm['zhp6'])))))))))

## overall zone starts

Zone starts of each player has been calculated only for his team being home or away for the season, since home zone start value and visitor zone start value were used. The **total zone starts** of each player is the total of zone starts he participated for a whole season. Thus, the sum of both home and away zone starts.

- create a variable will add up the home zone start value and away zone start value for all players of a given team, that played in position 1.

In [219]:
dm['zplyr1'] = np.where(dm['HTeamCode'] == dm['VTeamCode'], dm['tzhp1'] + dm['tzvp1'],
                       (np.where(dm['VTeamCode'] == dm['HTeamCode'], dm['tzvp1'] + dm['tzhp1'], 0)))

- create a variable will add up the home zone start value and away zone start value for all players of a given team, that played in position 2.

In [220]:
dm['zplyr2'] = np.where(dm['HTeamCode'] == dm['VTeamCode'], dm['tzhp2'] + dm['tzvp2'],
                       (np.where(dm['VTeamCode'] == dm['HTeamCode'], dm['tzvp2'] + dm['tzhp2'], 0)))

- create a variable will add up the home zone start value and away zone start value for all players of a given team, that played in position 3.

In [221]:
dm['zplyr3'] = np.where(dm['HTeamCode'] == dm['VTeamCode'], dm['tzhp3'] + dm['tzvp3'],
                       (np.where(dm['VTeamCode'] == dm['HTeamCode'], dm['tzvp3'] + dm['tzhp3'], 0)))

- create a variable will add up the home zone start value and away zone start value for all players of a given team, that played in position 4.

In [222]:
dm['zplyr4'] = np.where(dm['HTeamCode'] == dm['VTeamCode'], dm['tzhp4'] + dm['tzvp4'],
                       (np.where(dm['VTeamCode'] == dm['HTeamCode'], dm['tzvp4'] + dm['tzhp4'], 0)))

- create a variable will add up the home zone start value and away zone start value for all players of a given team, that played in position 5.

In [223]:
dm['zplyr5'] = np.where(dm['HTeamCode'] == dm['VTeamCode'], dm['tzhp5'] + dm['tzvp5'],
                       (np.where(dm['VTeamCode'] == dm['HTeamCode'], dm['tzvp5'] + dm['tzhp5'], 0)))

- create a variable will add up the home zone start value and away zone start value for all players of a given team, that played in position 6.

In [224]:
dm['zplyr6'] = np.where(dm['HTeamCode'] == dm['VTeamCode'], dm['tzhp6'] + dm['tzvp6'],
                       (np.where(dm['VTeamCode'] == dm['HTeamCode'], dm['tzvp6'] + dm['tzhp6'], 0)))                            

## allocate players per position to forward lines and defensive pairings

- If the zone start variable of a player for a given team is the highest in that specific position, it indicates that he participated in the most offensive zone starts. These skaters will be identified as **first line forwards and top defenisive pairing**. 

- If the zone start variable of a player is the second highest in that specific position, it indicates that the given skater participated in the second most offensive zone starts. These skaters will be identified as **second line forwards and second defenisive pairing**. 

- If the zone start variable of a player is the third highest in that specific position, it indicates that the given skater participated in the third most offensive zone starts. These skaters will be identified as **third line forwards and bottom defenisive pairing**. 

- If the zone start variable of a player is the lowest in that specific position, it indicates that the given skater participated in the least offensive zone starts. These skaters will be identified as **fourth line forwards**. 

### a) visitor team

- generate a variable that will allocate all players in position 1 to their respectful line. Position 1 is the **centre** position of forward lines. If **total zone start > 2**, player is assinged to the **top line**. If **total zone start between 2 and 0**, player is assinged to the **2nd line**. If **total zone start between 0 and -2**, player is assigned to the **3rd line**. Finally, if **total zone start less than -2**, player assigned to **4th line**.  

In [225]:
conditions = [ dm['tzvp1'] > 2, (dm['tzvp1'] <= 2) & (dm['tzvp1']> 0), (dm['tzvp1'] <= 0) & (dm['tzvp1']> -2), dm['tzvp1'] <= -2 ]
choices     = [ 1, 2, 3, 4 ]

dm['vc'] = np.select(conditions, choices, default=np.nan)

- generate a variable that will allocate all players in position 2 to their respectful line. Position 2 is the **right wing** position of forward lines. If **total zone start > 2**, player is assinged to the **top line**. If **total zone start between 2 and 0**, player is assinged to the **2nd line**. If **total zone start between 0 and -2**, player is assigned to the **3rd line**. Finally, if **total zone start less than -2**, player assigned to **4th line**.  

In [226]:
conditions = [ dm['tzvp2'] > 2, (dm['tzvp2'] <= 2) & (dm['tzvp2']> 0), (dm['tzvp2'] <= 0) & (dm['tzvp2']> -2), dm['tzvp2'] <= -2 ]
choices     = [ 1, 2, 3, 4 ]

dm['vrw'] = np.select(conditions, choices, default=np.nan)

- generate a variable that will allocate all players in position 3 to their respectful line. Position 3 is the **left wing** position of forward lines. If **total zone start > 2**, player is assinged to the **top line**. If **total zone start between 2 and 0**, player is assinged to the **2nd line**. If **total zone start between 0 and -2**, player is assigned to the **3rd line**. Finally, if **total zone start less than -2**, player assigned to **4th line**.  

In [227]:
conditions = [ dm['tzvp3'] > 2, (dm['tzvp3'] <= 2) & (dm['tzvp3']> 0), (dm['tzvp3'] <= 0) & (dm['tzvp3']> -2), dm['tzvp3'] <= -2 ]
choices     = [ 1, 2, 3, 4 ]

dm['vlw'] = np.select(conditions, choices, default=np.nan)

- generate a variable that will allocate all players in position 4 to their respectful line. Position 4 is the **right defense** position. If **total zone start > 3**, player is assinged to the **top defense pairing**. If **total zone start between 3 and 0**, player is assinged to the **2nd defense pairng**. Finally, if **total zone start less than 0**, player assigned to **3rd defense pairing**.   

In [228]:
conditions = [ dm['tzvp4'] > 3, (dm['tzvp4'] <= 3) & (dm['tzvp4']> 0), dm['tzvp4'] <= 0 ]
choices     = [ 1, 2, 3]

dm['vdr'] = np.select(conditions, choices, default=np.nan)

- generate a variable that will allocate all players in position 5 to their respectful line. Position 5 is the **left defense** position. If **total zone start > 3**, player is assinged to the **top defense pairing**. If **total zone start between 3 and 0**, player is assinged to the **2nd defense pairng**. Finally, if **total zone start less than 0**, player assigned to **3rd defense pairing**.  

In [229]:
conditions = [ dm['tzvp5'] > 3, (dm['tzvp5'] <= 3) & (dm['tzvp5']> 0), dm['tzvp5'] <= 0 ]
choices     = [ 1, 2, 3]

dm['vdl'] = np.select(conditions, choices, default=np.nan)

- Position 6 is the goaltender position. No conditions necessary 

In [230]:
dm['vg'] = 1

### b) home team

- generate a variable that will allocate all players in position 5 to their respectful line. Position 5 is the **left defense** position. If **total zone start > 3**, player is assinged to the **top defense pairing**. If **total zone start between 3 and 0**, player is assinged to the **2nd defense pairng**. Finally, if **total zone start less than 0**, player assigned to **3rd defense pairing**. 

In [231]:
conditions = [ dm['tzhp1'] > 2, (dm['tzhp1'] <= 2) & (dm['tzhp1']> 0), (dm['tzhp1'] <= 0) & (dm['tzhp1']> -2), dm['tzhp1'] <= -2 ]
choices     = [ 1, 2, 3, 4 ]

dm['hc'] = np.select(conditions, choices, default=np.nan)

- generate a variable that will allocate all players in position 2 to their respectful line. Position 2 is the **right wing** position of forward lines. If **total zone start > 2**, player is assinged to the **top line**. If **total zone start between 2 and 0**, player is assinged to the **2nd line**. If **total zone start between 0 and -2**, player is assigned to the **3rd line**. Finally, if **total zone start less than -2**, player assigned to **4th line**.  

In [232]:
conditions = [ dm['tzhp2'] > 2, (dm['tzhp2'] <= 2) & (dm['tzhp2']> 0), (dm['tzhp2'] <= 0) & (dm['tzhp2']> -2), dm['tzhp2'] <= -2 ]
choices     = [ 1, 2, 3, 4 ]

dm['hrw'] = np.select(conditions, choices, default=np.nan)

- generate a variable that will allocate all players in position 3 to their respectful line. Position 3 is the **left wing** position of forward lines. If **total zone start > 2**, player is assinged to the **top line**. If **total zone start between 2 and 0**, player is assinged to the **2nd line**. If **total zone start between 0 and -2**, player is assigned to the **3rd line**. Finally, if **total zone start less than -2**, player assigned to **4th line**.  

In [233]:
conditions = [ dm['tzhp3'] > 2, (dm['tzhp3'] <= 2) & (dm['tzhp3']> 0), (dm['tzhp3'] <= 0) & (dm['tzhp3']> -2), dm['tzhp3'] <= -2 ]
choices     = [ 1, 2, 3, 4 ]

dm['hlw'] = np.select(conditions, choices, default=np.nan)

- generate a variable that will allocate all players in position 4 to their respectful line. Position 4 is the **right defense** position. If **total zone start > 3**, player is assinged to the **top defense pairing**. If **total zone start between 3 and 0**, player is assinged to the **2nd defense pairng**. Finally, if **total zone start less than 0**, player assigned to **3rd defense pairing**.   

In [234]:
conditions = [ dm['tzhp4'] > 3, (dm['tzhp4'] <= 3) & (dm['tzhp4']> 0), dm['tzhp4'] <= 0 ]
choices     = [ 1, 2, 3]

dm['hdr'] = np.select(conditions, choices, default=np.nan)

- generate a variable that will allocate all players in position 5 to their respectful line. Position 5 is the **left defense** position. If **total zone start > 3**, player is assinged to the **top defense pairing**. If **total zone start between 3 and 0**, player is assinged to the **2nd defense pairng**. Finally, if **total zone start less than 0**, player assigned to **3rd defense pairing**.  

In [235]:
conditions = [ dm['tzhp5'] > 3, (dm['tzhp5'] <= 3) & (dm['tzhp5']> 0), dm['tzhp5'] <= 0 ]
choices     = [ 1, 2, 3]

dm['hdl'] = np.select(conditions, choices, default=np.nan)

- Position 6 is the goaltender position. No conditions necessary 

In [236]:
dm['vg'] = 1